In [1]:
import pandas as pd
import sklearn as sk
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score, roc_curve, auc
from sklearn.model_selection import KFold,TimeSeriesSplit
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

import lightgbm as lgb

import gc
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
plt.style.use('default') #ggplot스타일 사용

In [3]:
plt.style.use('ggplot') #ggplot스타일 사용
#한글폰트 사용
plt.rc('font', family='nanumgothic')
plt.rc('axes', unicode_minus=False)

In [4]:
plt.style.use('dark_background') #ggplot스타일 사용

---
# 데이터 파싱
---

In [85]:
train = pd.read_csv('train_.csv',encoding='utf8')
train2 = pd.read_csv('PJT002_train.csv')
train3 = pd.read_csv('train_quantile95.csv',encoding = 'utf8')

train.drop(columns='Unnamed: 0',inplace = True)
train3.drop(columns='Unnamed: 0',inplace = True)

C:\Users\tlsal\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\tlsal\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [86]:
'''24개 , 이상치 및 결측치를 처리한 변수 추출'''
float_ls = list(set(train.select_dtypes('float64').columns.tolist())-set(train.select_dtypes('float64').columns.tolist()[3:137]))

integer_ls = train.select_dtypes('int64').columns.tolist()
outlier_ls = integer_ls+float_ls
outlier_ls = outlier_ls[1:]

'''결측치 평균값으로 채우기'''
for i in range(len(outlier_ls)):
    train2[outlier_ls[i]] = train2[outlier_ls[i]].fillna(np.mean(train2[outlier_ls[i]]))

---
# 1. Label Encoding
---

In [37]:
'''
log1 = 이상치처리 90% (train)
log2 = 이상치처리 x (train2)
log3 = 이상치처리 95% (train3)
'''
train = train2
sample_var = pd.DataFrame()

sample_ls = list(range(5000,len(train),5000)) + [len(train)]

'''라벨인코딩하기 전에 date변수는 후에 계절변수로도 하나 만들고 시계열변수로도 하나 만들어도 좋을 듯'''
'''모델링할 때 잠깐 제외'''
train.drop(columns ='dt_of_fr',inplace=True)

'''label_encoding
결측치는 0으로 치환 됨'''
for col in train.columns:
    if train[col].dtype == 'object':
        le = LabelEncoder()
        le.fit(list(train[col].astype(str).values))
        train[col] = le.transform(list(train[col].astype(str).values))
        
'''결측치 일단 -999로 치환 -> 결측치 예측하는 거 유찬이가 만들었을 때 판단'''
train.fillna(-999,inplace=True)
train.drop(columns='id',inplace=True)

for k in sample_ls:
    '''방법론별 정확한 비교를 위해서 데이터는 샘플을 뽑아서 하지 않기로'''
    print('\n sample : ' + str(k))
    data = train.sample(k)
    
    variable = 'fr_yn'

    x_train, x_test, y_train, y_test = train_test_split(data.drop(columns=variable), 
                                                        np.array(data[variable].tolist()), 
                                                        test_size=0.2, 
                                                        stratify=np.array(data[variable].tolist()), 
                                                        random_state=123456)

    NFOLDS = 5
    folds = KFold(n_splits=NFOLDS)
    y_train = pd.Series(y_train)

    columns = x_train.columns
    splits = folds.split(x_train, y_train)
    y_preds = np.zeros(x_test.shape[0])
    y_oof = np.zeros(x_train.shape[0])
    score = 0
    acc=0
    f1=0
    test_score = 0
    test_acc=0
    test_f1=0

    feature_importances = pd.DataFrame()
    feature_importances['feature'] = columns  

    for fold_n, (train_index, valid_index) in enumerate(splits):
        x_train2, x_valid = x_train[columns].iloc[train_index], x_train[columns].iloc[valid_index]
        y_train2, y_valid = y_train.iloc[train_index], y_train.iloc[valid_index]

        #dtrain = xgb.DMatrix(data=x_train2,label=y_train2)
        #dvalid = xgb.DMatrix(data=x_valid, label=y_valid)

        #모델 생성(classification, regressor)
        xgb_class = xgb.XGBClassifier(n_estimators=100,random_state=0,max_depth=5,eval_metric='auc',class_weight='balanced')
        #모델 학습 (train,valid)
        xgb_class.fit(x_train2,y_train2)

        feature_importances[f'fold_{fold_n + 1}'] = xgb_class.feature_importances_
        y_pred_valid = xgb_class.predict(x_valid)
        y_oof[valid_index] = y_pred_valid

        test_pred = xgb_class.predict(x_test)

        print(f"Fold {fold_n + 1} | AUC: {roc_auc_score(y_valid, y_pred_valid)}")
        print(f"Fold {fold_n + 1} | ACC: {accuracy_score(y_valid,y_pred_valid)}")
        print(f"Fold {fold_n + 1} | F1_Score: {f1_score(y_valid,y_pred_valid)}")
        print(f"\nTEST Fold {fold_n + 1} | AUC: {roc_auc_score(y_test, test_pred)}")
        print(f"TEST Fold {fold_n + 1} | ACC: {accuracy_score(y_test, test_pred)}")
        print(f"TEST Fold {fold_n + 1} | F1_Score: {f1_score(y_test, test_pred)}")

        score += roc_auc_score(y_valid, y_pred_valid) / NFOLDS
        acc += accuracy_score(y_valid,y_pred_valid) / NFOLDS
        f1 += f1_score(y_valid,y_pred_valid) / NFOLDS

        '''test_fold'''
        test_score += roc_auc_score(y_test, test_pred) / NFOLDS
        test_acc += accuracy_score(y_test, test_pred) / NFOLDS
        test_f1 += f1_score(y_test, test_pred) / NFOLDS

        #y_preds += xgb_class.predict(x_test) / NFOLDS
        #y_test와의 성능 비교는 auc를 평가척도로 놓았기 때문에 그대로 y_test와 auc로 성능을 검증하면 됨
        #근데 의문은 1/0으로 binary로 결과치가 도출되는데 왜 predict_proba로 예측을 하지 않지?

        #데이터 메모리를 최적화하면서 모델링 진행
        del x_train2, x_valid, y_train2, y_valid
        gc.collect()

    print(f"\nMean AUC = {score}")
    print(f"Mean ACC = {acc}")
    print(f"Mean F1_Score = {f1}")
    print(f"\nOut of folds AUC = {roc_auc_score(y_train, y_oof)}")
    print(f"Out of folds AUC = {accuracy_score(y_train,y_oof)}")
    print(f"Out of folds F1_Score = {f1_score(y_train,y_oof)}")
    print(f"\nMean TEST AUC = {test_score}")
    print(f"Mean TEST ACC = {test_acc}")
    print(f"Mean TEST F1_Score = {test_f1}")

    eval_score = [test_score,test_acc,test_f1]
    sample_var['sample_'+str(k)] = eval_score


 sample : 5000
Fold 1 | AUC: 0.6182956908642453
Fold 1 | ACC: 0.88625
Fold 1 | F1_Score: 0.34532374100719426

TEST Fold 1 | AUC: 0.6641584823929197
TEST Fold 1 | ACC: 0.906
TEST Fold 1 | F1_Score: 0.4719101123595506
Fold 2 | AUC: 0.6610853227232538
Fold 2 | ACC: 0.90125
Fold 2 | F1_Score: 0.4697986577181208

TEST Fold 2 | AUC: 0.6425277269502222
TEST Fold 2 | ACC: 0.899
TEST Fold 2 | F1_Score: 0.42285714285714293
Fold 3 | AUC: 0.6095308080089761
Fold 3 | ACC: 0.89125
Fold 3 | F1_Score: 0.3358778625954198

TEST Fold 3 | AUC: 0.6449176593599463
TEST Fold 3 | ACC: 0.897
TEST Fold 3 | F1_Score: 0.42458100558659223
Fold 4 | AUC: 0.6242485532895106
Fold 4 | ACC: 0.8775
Fold 4 | F1_Score: 0.37179487179487175

TEST Fold 4 | AUC: 0.6577822174091639
TEST Fold 4 | ACC: 0.901
TEST Fold 4 | F1_Score: 0.4530386740331492
Fold 5 | AUC: 0.5874009847998287
Fold 5 | ACC: 0.8675
Fold 5 | F1_Score: 0.2933333333333333

TEST Fold 5 | AUC: 0.6343310056387468
TEST Fold 5 | ACC: 0.897
TEST Fold 5 | F1_Score: 0

Fold 2 | AUC: 0.6503274240016593
Fold 2 | ACC: 0.8866071428571428
Fold 2 | F1_Score: 0.4395410414827891

TEST Fold 2 | AUC: 0.6467447008259907
TEST Fold 2 | ACC: 0.8937142857142857
TEST Fold 2 | F1_Score: 0.4346504559270517
Fold 3 | AUC: 0.646179394022194
Fold 3 | ACC: 0.8903571428571428
Fold 3 | F1_Score: 0.42830540037243947

TEST Fold 3 | AUC: 0.6508355368365446
TEST Fold 3 | ACC: 0.8935714285714286
TEST Fold 3 | F1_Score: 0.4419475655430712
Fold 4 | AUC: 0.6516188011588985
Fold 4 | ACC: 0.8892857142857142
Fold 4 | F1_Score: 0.43738656987295826

TEST Fold 4 | AUC: 0.6518683841072151
TEST Fold 4 | ACC: 0.8921428571428571
TEST Fold 4 | F1_Score: 0.44198078344419817
Fold 5 | AUC: 0.6401588340758988
Fold 5 | ACC: 0.8930357142857143
Fold 5 | F1_Score: 0.4190106692531523

TEST Fold 5 | AUC: 0.6526374569706783
TEST Fold 5 | ACC: 0.8942857142857142
TEST Fold 5 | F1_Score: 0.44610778443113774

Mean AUC = 0.6498147218164005
Mean ACC = 0.8908571428571428
Mean F1_Score = 0.4365614433785883

Out 

In [38]:
sample_var2_label = sample_var.T
sample_var2_label = sample_var2_label.rename(columns={0:'AUC',
                                                         1:'ACC',
                                                         2:'F1_Score'})

---
# 2. Frequency Encoding
---

In [61]:
'''
log1 = 이상치처리 90% (train)
log2 = 이상치처리 x (train2)
log3 = 이상치처리 95% (train3)
'''
train = train2
sample_var = pd.DataFrame()

sample_ls = list(range(5000,len(train),5000)) + [len(train)]

'''라벨인코딩하기 전에 date변수는 후에 계절변수로도 하나 만들고 시계열변수로도 하나 만들어도 좋을 듯'''
'''모델링할 때 잠깐 제외'''
train.drop(columns ='dt_of_fr',inplace=True)

'''frequency_encoding'''
for col in train.drop(columns='fr_yn').columns:
    if train[col].dtype == 'object':
        train[col] = train[col].map(train[col].value_counts(dropna=False))
                                    
le = LabelEncoder()
le.fit(list(train['fr_yn'].astype(str).values))
train['fr_yn'] = le.transform(list(train['fr_yn'].astype(str).values))

train.fillna(-999,inplace=True)
train.drop(columns='id',inplace=True)

for k in sample_ls:
    '''방법론별 정확한 비교를 위해서 데이터는 샘플을 뽑아서 하지 않기로'''
    print('\n sample : ' + str(k))
    data = train.sample(k)
    
    variable = 'fr_yn'

    x_train, x_test, y_train, y_test = train_test_split(data.drop(columns=variable), 
                                                        np.array(data[variable].tolist()), 
                                                        test_size=0.2, 
                                                        stratify=np.array(data[variable].tolist()), 
                                                        random_state=123456)

    NFOLDS = 5
    folds = KFold(n_splits=NFOLDS)
    y_train = pd.Series(y_train)

    columns = x_train.columns
    splits = folds.split(x_train, y_train)
    y_preds = np.zeros(x_test.shape[0])
    y_oof = np.zeros(x_train.shape[0])
    score = 0
    acc=0
    f1=0
    test_score = 0
    test_acc=0
    test_f1=0

    feature_importances = pd.DataFrame()
    feature_importances['feature'] = columns  

    for fold_n, (train_index, valid_index) in enumerate(splits):
        x_train2, x_valid = x_train[columns].iloc[train_index], x_train[columns].iloc[valid_index]
        y_train2, y_valid = y_train.iloc[train_index], y_train.iloc[valid_index]

        #dtrain = xgb.DMatrix(data=x_train2,label=y_train2)
        #dvalid = xgb.DMatrix(data=x_valid, label=y_valid)

        #모델 생성(classification, regressor)
        xgb_class = xgb.XGBClassifier(n_estimators=100,random_state=0,max_depth=5,eval_metric='auc',class_weight='balanced')
        #모델 학습 (train,valid)
        xgb_class.fit(x_train2,y_train2)

        feature_importances[f'fold_{fold_n + 1}'] = xgb_class.feature_importances_
        y_pred_valid = xgb_class.predict(x_valid)
        y_oof[valid_index] = y_pred_valid

        test_pred = xgb_class.predict(x_test)

        print(f"Fold {fold_n + 1} | AUC: {roc_auc_score(y_valid, y_pred_valid)}")
        print(f"Fold {fold_n + 1} | ACC: {accuracy_score(y_valid,y_pred_valid)}")
        print(f"Fold {fold_n + 1} | F1_Score: {f1_score(y_valid,y_pred_valid)}")
        print(f"\nTEST Fold {fold_n + 1} | AUC: {roc_auc_score(y_test, test_pred)}")
        print(f"TEST Fold {fold_n + 1} | ACC: {accuracy_score(y_test, test_pred)}")
        print(f"TEST Fold {fold_n + 1} | F1_Score: {f1_score(y_test, test_pred)}")

        score += roc_auc_score(y_valid, y_pred_valid) / NFOLDS
        acc += accuracy_score(y_valid,y_pred_valid) / NFOLDS
        f1 += f1_score(y_valid,y_pred_valid) / NFOLDS

        '''test_fold'''
        test_score += roc_auc_score(y_test, test_pred) / NFOLDS
        test_acc += accuracy_score(y_test, test_pred) / NFOLDS
        test_f1 += f1_score(y_test, test_pred) / NFOLDS

        #y_preds += xgb_class.predict(x_test) / NFOLDS
        #y_test와의 성능 비교는 auc를 평가척도로 놓았기 때문에 그대로 y_test와 auc로 성능을 검증하면 됨
        #근데 의문은 1/0으로 binary로 결과치가 도출되는데 왜 predict_proba로 예측을 하지 않지?

        #데이터 메모리를 최적화하면서 모델링 진행
        del x_train2, x_valid, y_train2, y_valid
        gc.collect()

    print(f"\nMean AUC = {score}")
    print(f"Mean ACC = {acc}")
    print(f"Mean F1_Score = {f1}")
    print(f"\nOut of folds AUC = {roc_auc_score(y_train, y_oof)}")
    print(f"Out of folds AUC = {accuracy_score(y_train,y_oof)}")
    print(f"Out of folds F1_Score = {f1_score(y_train,y_oof)}")
    print(f"\nMean TEST AUC = {test_score}")
    print(f"Mean TEST ACC = {test_acc}")
    print(f"Mean TEST F1_Score = {test_f1}")

    eval_score = [test_score,test_acc,test_f1]
    sample_var['sample_'+str(k)] = eval_score


 sample : 5000
Fold 1 | AUC: 0.6131256384065372
Fold 1 | ACC: 0.8875
Fold 1 | F1_Score: 0.3382352941176471

TEST Fold 1 | AUC: 0.6395640005891884
TEST Fold 1 | ACC: 0.884
TEST Fold 1 | F1_Score: 0.4020618556701031
Fold 2 | AUC: 0.6541718924896495
Fold 2 | ACC: 0.89375
Fold 2 | F1_Score: 0.4516129032258064

TEST Fold 2 | AUC: 0.6401347768448961
TEST Fold 2 | ACC: 0.885
TEST Fold 2 | F1_Score: 0.40414507772020725
Fold 3 | AUC: 0.6123437475307764
Fold 3 | ACC: 0.89625
Fold 3 | F1_Score: 0.3464566929133858

TEST Fold 3 | AUC: 0.6637207247017234
TEST Fold 3 | ACC: 0.896
TEST Fold 3 | F1_Score: 0.4583333333333333
Fold 4 | AUC: 0.6524523849654474
Fold 4 | ACC: 0.8975
Fold 4 | F1_Score: 0.44594594594594594

TEST Fold 4 | AUC: 0.6499116217410517
TEST Fold 4 | ACC: 0.89
TEST Fold 4 | F1_Score: 0.4270833333333333
Fold 5 | AUC: 0.6507905138339921
Fold 5 | ACC: 0.87875
Fold 5 | F1_Score: 0.4327485380116959

TEST Fold 5 | AUC: 0.629750331418471
TEST Fold 5 | ACC: 0.885
TEST Fold 5 | F1_Score: 0.385

Fold 2 | AUC: 0.6468015399608789
Fold 2 | ACC: 0.89
Fold 2 | F1_Score: 0.432780847145488

TEST Fold 2 | AUC: 0.6478196651160222
TEST Fold 2 | ACC: 0.8917142857142857
TEST Fold 2 | F1_Score: 0.43432835820895527
Fold 3 | AUC: 0.6477496907933069
Fold 3 | ACC: 0.89
Fold 3 | F1_Score: 0.4296296296296296

TEST Fold 3 | AUC: 0.6565136477205277
TEST Fold 3 | ACC: 0.8931428571428571
TEST Fold 3 | F1_Score: 0.4516129032258065
Fold 4 | AUC: 0.6537998066961279
Fold 4 | ACC: 0.8964285714285715
Fold 4 | F1_Score: 0.4486692015209125

TEST Fold 4 | AUC: 0.6512857130308092
TEST Fold 4 | ACC: 0.8937142857142857
TEST Fold 4 | F1_Score: 0.4431137724550898
Fold 5 | AUC: 0.6487462422634837
Fold 5 | ACC: 0.8933928571428571
Fold 5 | F1_Score: 0.4362606232294618

TEST Fold 5 | AUC: 0.6491567665111002
TEST Fold 5 | ACC: 0.8924285714285715
TEST Fold 5 | F1_Score: 0.43764002987303957

Mean AUC = 0.650509334905847
Mean ACC = 0.8924285714285713
Mean F1_Score = 0.440129891674092

Out of folds AUC = 0.650522061395934

In [62]:
sample_var2_freq = sample_var.T
sample_var2_freq = sample_var2_freq.rename(columns={0:'AUC',
                                                         1:'ACC',
                                                         2:'F1_Score'})

In [72]:
sample_var2_freq.to_csv('freq_encoding_compare.csv',index=True)
sample_var2_label.to_csv('label_encoding_compare.csv',index=True)

---
# 3. Target Encoding
---

---
#### 3-1. target based probability encoding
---

---
#### 3-2. target mean encoding
---

In [ ]:
def target_encoding():
    df[col]

In [ ]:
train2['bldng_us'].value_counts(dropna=)

---
# 4. 결과비교
---

In [63]:
sample_var2_label

,AUC,ACC,F1_Score
sample_5000,0.648743,0.900000,0.435402
sample_10000,0.632752,0.888000,0.400576
sample_15000,0.648228,0.895467,0.429709
sample_20000,0.636792,0.889600,0.408640
sample_25000,0.648391,0.888440,0.430159
sample_30000,0.646407,0.893067,0.432343
sample_35000,0.650704,0.893343,0.441396
sample_40000,0.661202,0.897050,0.462812
sample_45000,0.643553,0.890733,0.423024
sample_50000,0.646733,0.890320,0.428252


In [64]:
sample_var2_freq

,AUC,ACC,F1_Score
sample_5000,0.644616,0.888000,0.415330
sample_10000,0.643766,0.890200,0.423184
sample_15000,0.632243,0.889867,0.401457
sample_20000,0.628693,0.887800,0.392495
sample_25000,0.652602,0.894960,0.443646
sample_30000,0.641076,0.892767,0.418497
sample_35000,0.650498,0.892829,0.440518
sample_40000,0.651186,0.893900,0.443844
sample_45000,0.652933,0.892889,0.444202
sample_50000,0.643405,0.892060,0.425347
